In [ ]:
!pip install mlflow boto3 dagshub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/1

In [ ]:
## step 2: set up the mlflow tracking server
import dagshub
import mlflow
dagshub.init(repo_owner='codex03080', repo_name='reddit-sentiement-analysis', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=5cb2ac64-e1eb-46b4-97f0-de75febbf491&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=650771c37c1d0d08bcae387cbbf526d47ab75879c9fab5d1e4afa1393221bade




Accessing as codex03080

Initialized MLflow to track repo "codex03080/reddit-sentiement-analysis"

Repository codex03080/reddit-sentiement-analysis initialized!

In [ ]:
## Step 2: set or create an experiment
mlflow.set_experiment("Ex 4 - Handling imbalanced data")

2026/02/10 08:03:28 INFO mlflow.tracking.fluent: Experiment with name 'Ex 4 - Handling imbalanced data' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/b3b6d94e389d4d54b1b14cdaba2bd17e', creation_time=1770710608509, experiment_id='4', last_update_time=1770710608509, lifecycle_stage='active', name='Ex 4 - Handling imbalanced data', tags={}>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [ ]:
df = pd.read_csv("/content/reddit_preprocessing.csv").dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
print("Imbalanced-learn libraries imported.")

Imbalanced-learn libraries imported.


In [ ]:
## Step 1: Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
  ngram_range = (1,2)
  max_features = 10000

  ## Step 2: Train Test Split
  X_train,X_test,y_train,y_test = train_test_split(df['clean_comment'],df['category'],test_size=0.2,random_state=42)

  ## Step 3: Vectorization using Tf-idf,fit on training data only
  vectorizer = TfidfVectorizer(ngram_range=ngram_range,max_features=max_features)
  X_train_vec = vectorizer.fit_transform(X_train)
  X_test_vec = vectorizer.transform(X_test)

  ## Step 4: Handle imbalanced data
  if imbalance_method == 'class_weights':
    class_weight = 'balanced'
  else:
    class_weight = None

    ## Resampling techiques
    if imbalance_method == 'oversampling':
      smote = SMOTE(random_state=42)
      X_train_vec,y_train = smote.fit_resample(X_train_vec,y_train)
    elif imbalance_method == 'adasyn':
      adasyn = ADASYN(random_state=42)
      X_train_vec,y_train = adasyn.fit_resample(X_train_vec,y_train)
    elif imbalance_method == 'undersampling':
      rus = RandomUnderSampler(random_state=42)
      X_train_vec,y_train = rus.fit_resample(X_train_vec,y_train)
    elif imbalance_method == 'smote_enn':
      smote_enn = SMOTEENN(random_state=42)
      X_train_vec,y_train = smote_enn.fit_resample(X_train_vec,y_train)

  ## step 5: Define and train a random forest model
  with mlflow.start_run() as run:
    ## set tag for experiment and run
    mlflow.set_tag("mlflow.runName",f"Imbalance_{imbalance_method}_RandomForest_Tfidf_bigrams")
    mlflow.set_tag("experiment_type","imbalanced_handling")
    mlflow.set_tag("model_type","RandomForestClassifier")

    ## Add a description
    mlflow.set_tag("description",f"RandomForest with TF-IDF bigrams,imbalance handling method={imbalance_method}")

    ## Log vectorizer params
    mlflow.log_param("vectorizer_type","Tf-idf")
    mlflow.log_param("ngram_range",ngram_range)
    mlflow.log_param("vectorizer_max_features",max_features)

    ## Log Random Forest parameters
    n_estimators = 200
    max_depth = 15

    ## Intialization and train the model
    model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,random_state=42,class_weight=class_weight)
    model.fit(X_train_vec, y_train) # Train the model

    y_pred = model.predict(X_test_vec) # Make predictions

    ## Log accuracy
    accuracy = accuracy_score(y_test,y_pred)
    mlflow.log_metric("accuracy",accuracy)

    ## Log classification report
    classification_rep = classification_report(y_test,y_pred,output_dict=True)
    for label,metrics in classification_rep.items():
      if isinstance(metrics,dict):
        for metric,value in metrics.items():
          mlflow.log_metric(f"{label}_{metric}",value)

    ## Log confusion matrix
    conf_matrix = confusion_matrix(y_test,y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(conf_matrix,annot=True,fmt='d',cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.savefig(f'confusion_matrix_{imbalance_method}.png')
    mlflow.log_artifact(f'confusion_matrix_{imbalance_method}.png')
    plt.close()

    ## Log the model
    mlflow.sklearn.log_model(model,f"Random_forest_model_tfidf_bigram_imbalance_{imbalance_method}")

## step 7: run experiment for different imabalanced method
imbalance_methods = ['class_weights','oversampling','adasyn','undersampling','smote_enn']
for method in imbalance_methods:
  run_imbalanced_experiment(method)

2026/02/10 08:38:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run Imbalance_class_weights_RandomForest_Tfidf_bigrams at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/4/runs/89db12179f4c486d86aaef01a7330f4c
🧪 View experiment at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/4


2026/02/10 08:39:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run Imbalance_oversampling_RandomForest_Tfidf_bigrams at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/4/runs/3a37bde9fa154a55b88cb85415026e97
🧪 View experiment at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/4


2026/02/10 08:40:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run Imbalance_adasyn_RandomForest_Tfidf_bigrams at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/4/runs/4307d90a83884616bb2d2ede684561cd
🧪 View experiment at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/4


2026/02/10 08:41:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run Imbalance_undersampling_RandomForest_Tfidf_bigrams at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/4/runs/99c5d15a4a114b2b9d7ef01ef80f66fb
🧪 View experiment at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/4


2026/02/10 08:43:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run Imbalance_smote_enn_RandomForest_Tfidf_bigrams at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/4/runs/4198aefefdc6425c8da6867d5076958c
🧪 View experiment at: https://dagshub.com/codex03080/reddit-sentiement-analysis.mlflow/#/experiments/4
